In [1]:
# Example-5-MC-GPA-IRS-CVA
# Author: Matthew Dixon
# Version: 1.0 (28.4.2020)
# License: MIT
# Email: matthew.dixon@iit.edu
# Notes: tested on Mac OS X running Python 3.6.9 with the following packages:
# scikit-learn=0.22.1, numpy=1.18.1, matplotlib=3.1.3
# Citation: Please cite the following reference if this notebook is used for research purposes:
# Dixon M.F., Halperin I. and P. Bilokon, Machine Learning in Finance: From Theory to Practice, Springer Graduate # Textbook Series, 2020.


# Overview
This example demonstrates the application of Gaussian processes to CVA modeling on a counterparty portfolio of IRS contracts with 11 currencies and 10 FX processes. The notebook simulates the GP mark-to-market cube of the portfolio, and compares the Expected Positive Exposure when using a GP derivative pricing model versus a BS pricing model and calculate the CVA${}_0$.

## Data preparation
In the data, the interest rates appear first and then the FX processes, so that if $0 \leq j \leq n_r-1$, where $n_r$ is the number of rates, then the $j^{~th}$ process is the $j^{~th}$ interest rate and, ignoring $j=0$, the $(n_r+j-1)^{th}$ process is the FX process whose foreign currency is the $j^{~th}$ one.

For example, in the case of 3 currencies (and therefore 2 FX rates) we would have a spatial arrangement like this:

|  rate 0  |  rate 1  |  rate 2  |  FX 0  |  FX 1  |


In the rates cube, $\left[i, j, k\right]$ refers to the $i^{~th}$ coarse time step, $j^{~th}$ interest rate process if $j\leq n_r$, or the $({j-n_r})^{~th}$ FX process otherwise, and the $k^{~th}$ Monte Carlo scenario.

## Data Download

Please download the zipped .npz files from the following link, unzip them, and put them directly into the data folder `ML_Finance_Codes/data/`

https://www.dropbox.com/sh/2dd1aida38nanwl/AACaCt1fXX2xr6I5T4B9U3_1a?dl=0

In [2]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn import gaussian_process
from sklearn.gaussian_process.kernels import ConstantKernel, RBF, Matern

%matplotlib inline

In [5]:
mtm_irs = np.load('../../data/mtm_irs.npz')

In [6]:
rates_and_fx = np.load('../../data/rates_and_fx.npz')

In [7]:
irs_params = np.load('../../data/irs_params.npz')

In [8]:
rate_params = np.load('../../data/rate_params.npz')

# Aggregate underlying data together

The aggregation of the data is as follows:
- Select counterparty portfolio (1 of 5)
- Select a IRS contract `i`
- Select times t = ${t_j}$
- Get M simulated prices from `mtm_irs[t, i, M]`

- Get currency from `irs_params['arr_0'][i]`
- Get foreign rate and FX if currency $\neq$ 0, else get domestic rate

In [ ]:
cpty_idx = 0
port_idx = []
for i in range(len(irs_params['arr_0'])):
    if (irs_params['arr_0'][i][5] == cpty_idx):
        port_idx.append(i)

In [ ]:
port_idx = np.array(port_idx)
T = np.shape(mtm_irs['arr_0'])[0]

In [ ]:
np.shape(mtm_irs['arr_0'])

### Reset dates

In [ ]:
reset_idx = (np.arange(101) - 1) // 5*5

In [ ]:
M = 2000

Construct IRS mappings to rates

In [ ]:
irs_map = {}
for i in port_idx:
    ccy = irs_params['arr_0'][i][6]
    if ccy == 0:
        irs_map[i] = [0] # domestic rate
    else:
        irs_map[i] = [ccy, ccy+10] # foreign rate + FX rate

In [ ]:
mtm = []
rates_fx = []
prev_rates_fx = []
for t in np.arange(5, T, 10):     
    mtm.append(mtm_irs['arr_0'][t, port_idx,:2*M])
    
    prev_rates_fx.append(rates_and_fx['arr_0'][t-5, :, :2*M].T) # previous reset date
    
    rates_fx.append(rates_and_fx['arr_0'][t, :, :2*M].T)

nt = len(np.arange(1, T, 10))

This function can be used to sample the input space with the option method

In [ ]:
def stratified_sampling(ar, M):
    quantiles = [0, 0.25, 0.5, 0.75, 1]
    num_buckets = len(quantiles) - 1
    q = np.quantile(ar, quantiles, axis=0)
    
    idx = np.array([], dtype=int)
    sum_ = 0
    dim = np.shape(ar)[1]
    num_samples = int(M / (num_buckets**dim))

    if dim == 2: # domestic
        for i in range(num_buckets):
            for j in range(num_buckets):
                idx_= np.where((ar[:,0] >= q[i,0]) & (ar[:,0] <= q[i+1,0]) 
                             & (ar[:,1] >= q[j,1]) & (ar[:,1] <= q[j+1,1]))
                
                if len(idx_[0]) >  num_samples:
                    idx__ = idx_[0][:num_samples]
                else: # oversample
                    idx__ = np.random.choice(idx_[0], num_samples) # sample with replacement 
     
                sum_+= len(idx__) 
                idx = np.append(idx, idx__)
    else: 
        counter = 0
        for i in range(num_buckets):
            for j in range(num_buckets):
                for k in range(num_buckets):
                    idx_= np.where((ar[:, 0] >= q[i, 0]) & (ar[:, 0] <= q[i+1, 0]) 
                                 & (ar[:, 1] >= q[j, 1]) & (ar[:, 1] <= q[j+1, 1]) 
                                 & (ar[:, 2] >= q[k, 2]) & (ar[:, 2] <= q[k+1, 2]))
                    if len(idx_[0]) > num_samples:
                        idx__ = idx_[0][:num_samples]
                        idx = np.append(idx, idx__)
                        sum_ += len(idx__)
                    elif len(idx_[0]) > 0: # oversample
                        idx__ = np.random.choice(idx_[0], num_samples) # sample with replacement 
                        
                        idx = np.append(idx,idx__)
                        sum_ += len(idx__)
                    else:
                        counter += 1 
                        print(i, j, k, 0, counter)
    
    # ensure that minimums and maximums are in the training set
    idx_min = np.argmin(ar, axis=0)
    idx_max = np.argmax(ar, axis=0)
    
    idx[0:dim] = idx_min
    idx[dim:2*dim] = idx_max
    return idx

## GP kernel specification

In [ ]:
sk_kernel_matern = Matern(length_scale=1.0, nu=0.5, length_scale_bounds=(0.01, 10000))
sk_kernel_rbf = RBF(length_scale=1.0, length_scale_bounds=(0.01, 10000))

In [ ]:
portfolio = {}
for idx in port_idx:
    portfolio[idx] = {'GPs':np.array([0]*nt, dtype=object), 'weight':1, 
                      'min':np.array([0]*nt, dtype=object), 
                      'max':np.array([0]*nt, dtype=object), 
                      'min_sc_x':np.array([0]*nt, dtype=object),
                      'max_sc_x':np.array([0]*nt, dtype=object),
                      'min_sc_y':np.array([0]*nt, dtype=object), 
                      'max_sc_y':np.array([0]*nt, dtype=object)
                     }

gps = []
for i in range(nt):
    for j in range(20):
        n_rates = len(irs_map[port_idx[j]])
        if i > 0:
            rates_ = np.zeros([2*M, n_rates+1])  
            rates_[:, 0] = prev_rates_fx[i][:, irs_map[port_idx[j]][0]]
            rates_[:, 1:] = rates_fx[i][:, irs_map[port_idx[j]]] 
        else:
            rates_ = np.zeros([2*M,n_rates])  
            rates_ = rates_fx[i][:, irs_map[port_idx[j]]] 
        idx = stratified_sampling(rates_, M)
        print(len(idx))  
        gp = gaussian_process.GaussianProcessRegressor(kernel=sk_kernel_matern, n_restarts_optimizer=20)
        
        # Rescale x and y to the unit interval
        rates_sc_ = (rates_ - np.min(rates_, axis=0)) / (np.max(rates_, axis=0) - np.min(rates_, axis=0) + 1e-16)
        irs_prices = mtm[i][j, :]
          
        if np.max(irs_prices) - np.min(irs_prices) != 0:  
            irs_prices_sc_ = (irs_prices - np.min(irs_prices)) / (np.max(irs_prices) - np.min(irs_prices))
        else:
            irs_prices_sc_ = irs_prices - np.min(irs_prices)
        
        portfolio[port_idx[j]]['GPs'][i] = gp.fit(rates_sc_[idx, :], irs_prices_sc_[idx]) 
        portfolio[port_idx[j]]['min'][i] = np.min(rates_[idx, :], axis=0)
        portfolio[port_idx[j]]['max'][i] = np.max(rates_[idx, :], axis=0)
        portfolio[port_idx[j]]['min_sc_x'][i] = np.min(rates_, axis=0)
        portfolio[port_idx[j]]['max_sc_x'][i] = np.max(rates_, axis=0)
        portfolio[port_idx[j]]['min_sc_y'][i] = np.min(irs_prices)
        portfolio[port_idx[j]]['max_sc_y'][i] = np.max(irs_prices)
          
        print(i, j)

In [ ]:
# CVA simulation using the credit default model described in the above reference.
def CVA_simulation(sim_params, model_params, def_model):
    
    M        = sim_params['M']        # number of paths
    nt       = sim_params['nt']       # number of exposure dates 
    r        = model_params['r']
    T        = model_params['T'] 
    t0       = model_params['t0'] 
    haz_rate = model_params['lambda']
    n_instruments = model_params['size']
    
    pi = {}
    pi['tilde'] = np.array([0.0]*nt*M, dtype='float32').reshape(nt, M)     # GP portfolio value
    pi['exact'] = np.array([0.0]*nt*M, dtype='float32').reshape(nt, M)     # BS portfolio value
    pi['tilde_var'] = np.array([0.0]*nt*M, dtype='float32').reshape(nt, M) # GP portfolio variance
    dPD = np.array([0.0]*nt*M, dtype='float32').reshape(nt, M)             # default probabilities
    dt = T/nt 
    
    for i in range(nt): # len(mtm) 
      
        pred_ = 0
        var_ = 0
        obs_ = 0 

        for j in range(n_instruments): # loop over instruments
            idx = np.random.choice(range(2*M), M)
            irs_prices = mtm[i][j, idx] # test samples
            rates = rates_fx[i][:, irs_map[port_idx[j]]]  # test samples
            prev_rates = prev_rates_fx[i][:, irs_map[port_idx[j]][0]]  # test samples
               
            n_rates = np.shape(rates)[1]
            
            key = port_idx[j]
            
            # Avoid simulated rates/FX breaching boundaries of training set
            if i > 0:
                dim = n_rates + 1
                test_rates = np.zeros([M, dim])  
                test_rates[:, 0] = prev_rates[idx]
                test_rates[:, 1:] = rates[idx, :] 
            else:
                dim = n_rates  
                test_rates = np.zeros([M, dim])  
                test_rates = rates[idx, :] 
                
            # Rescale x and y to the unit interval
            for k in range(dim):
                idx_min = np.where(test_rates[:, k] < portfolio[key]['min'][i][k])
                idx_max = np.where(test_rates[:, k] > portfolio[key]['max'][i][k])
                if len(idx_min[0]) > 0:
                    test_rates[idx_min, k] = portfolio[key]['min'][i][k]
                if len(idx_max[0]) > 0:
                    test_rates[idx_max, k] = portfolio[key]['max'][i][k]
            min_ = portfolio[port_idx[j]]['min_sc_x'][i]
            max_ = portfolio[port_idx[j]]['max_sc_x'][i]
            
            test_rates_sc = (test_rates - min_) / (max_ - min_)
            
            pred_sc, std = portfolio[key]['GPs'][i].predict(test_rates_sc, return_std=True) 
            min_y_ = portfolio[port_idx[j]]['min_sc_y'][i]
            max_y_ = portfolio[port_idx[j]]['max_sc_y'][i]
            
            pred = min_y_ + pred_sc * (max_y_ - min_y_)
            std *= (max_y_ - min_y_)
            pred_ += portfolio[key]['weight'] * pred
            var_ += (portfolio[key]['weight'] * std)**2 
            obs_ += portfolio[key]['weight'] * irs_prices

        pi['tilde'][i, :] = pred_
        pi['exact'][i, :] = obs_
        pi['tilde_var'][i, :] = var_ 
        
        # compute default probabilities  
        dPD[i, :]= haz_rate * np.exp(-haz_rate*dt)#gamma[i-1,m]*exp_factor

    CVA = {}
    CVA['tilde'] = 0
    CVA['exact'] = 0
    CVA['exact_up'] = 0
    CVA['exact_down'] = 0
    CVA['tilde_up'] = 0
    CVA['tilde_down'] = 0
      
    for i in range(nt): 
        time = i * dt
        mu_tilde = np.mean(dPD[i, :] * pi['tilde'][i, :]) * np.exp(-r*(time-t0)) * dt
        CVA['tilde'] += mu_tilde
        
        std_tilde_err = np.std(dPD[i, :] * pi['tilde'][i, :]) * np.exp(-r*(time-t0)) * dt/sqrt(M)
        CVA['tilde_up'] += mu_tilde + 2*std_tilde_err
        CVA['tilde_down'] += mu_tilde - 2*std_tilde_err
        
        mu_exact = np.mean(dPD[i, :] * pi['exact'][i, :]) * np.exp(-r*(time-t0))*dt
        CVA['exact'] += mu_exact
        
        std_exact_err = np.std(dPD[i, :] * pi['exact'][i, :]) * np.exp(-r*(time-t0)) * dt/sqrt(M)
        CVA['exact_up'] += mu_exact + 2*std_exact_err
        CVA['exact_down'] += mu_exact - 2*std_exact_err
        
    CVA['tilde'] *= (1-def_model['recovery'])
    CVA['tilde_up'] *= (1-def_model['recovery'])
    CVA['tilde_down'] *= (1-def_model['recovery'])
    CVA['exact'] *= (1-def_model['recovery'])
    CVA['exact_up'] *= (1-def_model['recovery'])
    CVA['exact_down'] *= (1-def_model['recovery'])
        
    return CVA, pi

In [ ]:
def_model = {}
def_model['recovery'] = 0.4 # recovery rate

sim_params = {}
sim_params['M'] = 1000   # Number of simulations  
sim_params['nt'] = 10     # Number of exposure dates
    
model_params= {}   
model_params['r'] = 0.02
model_params['T'] = 10.0 # Longest dated cash flow maturity in the portfolio 
model_params['t0'] = 0
model_params['lambda'] = 0.05 # constant hazard rate
model_params['size'] = 20 # number of instruments in portfolio

CVA_0, pi_0 = CVA_simulation(sim_params, model_params, def_model)

In [ ]:
CVA_0_err = 100 * (CVA_0['exact'] - CVA_0['tilde']) / CVA_0['exact']
print('% error in CVA_0:', CVA_0_err)

Compare the exact CVA, i.e. using BS prices, with the GP portfolio and provide the GP error bounds 

In [ ]:
CVA_0

In [ ]:
epe_gp = np.mean(pi_0['tilde'], axis=1)
epe_obs = np.mean(pi_0['exact'], axis=1)
epe_var = np.mean(pi_0['tilde_var'], axis=1)

Error plot in expected positive exposure

In [ ]:
plt.figure(figsize = (10, 6), facecolor='white', edgecolor='black')
plt.plot(0.1 + np.arange(10), epe_gp, color = 'red', label = 'GP Prediction')
plt.plot(0.1 + np.arange(10), epe_obs, color = 'black', label = 'Analytical Model')
plt.fill_between(0.1 + np.arange(10), (epe_gp - 2.0 * np.sqrt(epe_var)), (epe_gp + 2.0 * np.sqrt(epe_var)), color = 'grey', alpha=0.3)
plt.legend(loc = 'best', prop={'size':10})
plt.xlabel('time (years)')
plt.ylabel('EPE (Euro)');